In [7]:
from selenium import webdriver
from selenium.webdriver import ActionChains
import urllib
import time
import pandas as pd
from tqdm import trange, tqdm, tqdm_notebook
import numpy as np

In [2]:
driver = webdriver.Chrome('C:/Temp/chromedriver')

In [3]:
driver.get('https://gongu.copyright.or.kr/gongu/wrt/wrtCl/listWrtSound.do?menuNo=200020&pageIndex=1')

In [4]:
def pagein(mtitle, mtag1, mtag2, mtag3, titles, tags1, tags2, tags3):
    for title, tag1, tag2, tag3 in zip(titles, tags1, tags2, tags3):
        if (title.text == mtitle) & (tag1.text == mtag1) & (tag2.text == mtag2) & (tag3.text == mtag3):
            title.click()
            break

In [5]:
def downloadmp3(names, checkboxes):
    cnt = 0
    for name in names:
        if name.text[-3:] in ['mp3', 'wav']:
            checkboxes[cnt].click()
            driver.find_element_by_css_selector('#button_download').click()
            break
        cnt += 1

In [59]:
music = pd.read_csv('music_emotion.csv').iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,17]]
music = music[(music['emotion'] == '우울') | (music['emotion'] == '기쁨')]

In [60]:
music

,title,recommend,clicks,download,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,tag10,emotion
0,4. Keyboard Concerto No. 5 in F Minor BWV 1056...,22,5901,1794,명상,기타(GUITAR),EMOTIONAL,매우느리게,Jazz,드럼(DRUM),피아노(PIANO),콘트라베이스(CONTRABASS),ballad,NaN,우울
5,"9. Prelude, op.28-15(Rain Drop)_ 쇼팽 : 전주곡 15번 ...",12,4227,950,CHIC,세련,느리게,Jazz,드럼(DRUM),피아노(PIANO),콘트라베이스(CONTRABASS),ballad,NaN,NaN,우울
8,2. Frolicher Landmann_ 행복한 농부_ragtime,28,6296,2629,경쾌,ENERGETIC,빠르게,기타(GUITAR),Jazz,드럼(DRUM),피아노(PIANO),콘트라베이스(CONTRABASS),ragtime,NaN,기쁨
9,27. Kinderszenen Op.15 - VII. Traumerei_ 어린이의 ...,16,4349,1502,명상,EMOTIONAL,매우느리게,Jazz,드럼(DRUM),피아노(PIANO),콘트라베이스(CONTRABASS),ballad,NaN,NaN,우울
10,"97. Symphony No. 5 in Cminor, Op.67_ 교향곡 제5번 다...",2,2004,562,DARK,빠르게,오케스트라(ORCHESTRA),재분류,string,어두운,NaN,NaN,NaN,NaN,기쁨
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499,쪼각빗,0,445,229,기타,BRIGHT,경쾌,ENERGETIC,밝은,상큼,FRESH,신스(SYNTH),키보드(KEYBOARD),빠르게,기쁨
1504,고향의 봄,2,3753,583,BRIGHT,밝은,타악기(PERCUSSION),신스(SYNTH),CHIC,세련,느리게,키보드(KEYBOARD),월드뮤직,팝,우울
1519,가을,0,717,292,기타,BRIGHT,경쾌,ENERGETIC,밝은,상큼,FRESH,신스(SYNTH),키보드(KEYBOARD),빠르게,기쁨
1520,꽃밭,0,888,258,기타,BRIGHT,경쾌,ENERGETIC,밝은,상큼,FRESH,신스(SYNTH),키보드(KEYBOARD),빠르게,기쁨


In [63]:
driver.switch_to.window(driver.window_handles[0])

In [37]:
URL = 'https://gongu.copyright.or.kr/gongu/wrt/wrtCl/listWrtSound.do?menuNo=200020&pageIndex=1&searchWrd='
for mtitle, tag1, tag2, tag3 in tqdm(zip(list(music['title']), list(music['tag1']), list(music['tag2']), list(music['tag3']))):
    driver.get(URL + mtitle)
    time.sleep(2)
    titles = driver.find_elements_by_css_selector('ul > li > div:nth-child(1) > div.col-md-8 > span.tit > a')
    tags1 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(1)")
    tags2 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(2)")
    tags3 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(3)")
    pagein(mtitle, tag1, tag2, tag3, titles, tags1, tags2, tags3)
    time.sleep(1)
    try:
        driver.find_element_by_css_selector('#chkDownAt').click()
    except:
        print(mtitle)
        continue
    driver.find_element_by_css_selector('#downChkBtn').click()
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[-1])
    try:
        driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
    except:
        print(mtitle)
        driver.switch_to.window(driver.window_handles[0])
        continue        
    if driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span') == []:
        driver.switch_to.window(driver.window_handles[0])
        continue
    downloadmp3(driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span'),
                driver.find_elements_by_css_selector('#file_list > li > ul > li.input_chk'))
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(1)

2it [00:11,  5.17s/it]

9. Prelude, op.28-15(Rain Drop)_ 쇼팽 : 전주곡 15번 내림 라장조 작품번호 28-15 '빗방울'_ballad


9it [01:23,  9.52s/it]

101. Serenade No.13 in G Major K525 'Eine Kleins Nachtmusik'_ 세레나데 제 13번 사장조 '아이네 클라이네 나흐트 무지크'_string


10it [01:26,  7.54s/it]

...

Lyric Pieces, Op.38, No.4 'Halling' (Norwegian Dance)


262it [34:08,  6.33s/it]

Overture from "A Midsummer's Night Dream Incidental Music", Op. 61


270it [35:22,  7.86s/it]


ElementClickInterceptedException: Message: element click intercepted: Element <iframe id="dext5uploader_frame_dext5upload" title="DEXT5Upload dext5upload" frameborder="0" src="" style="width: 100%; height: 100%; border-width: 0px;" cd_frame_id_="25b7f8886fdb5a8923aa7f60b7236f08"></iframe> is not clickable at point (32, 141). Other element would receive the click: <div style="filter:alpha(opacity=50); opacity:0.5; -moz-opacity:0.5; -ms-filter:progid:DXImageTransform.Microsoft.Alpha(Opacity=50); -khtml-opacity: 0.5; background-color:black; z-index:1000; position:fixed; _position:absolute; left:0px; top:0px;overflow:hidden;width:100%;height:100%;"></div>
  (Session info: chrome=90.0.4430.212)


In [44]:
music[music['title'] == 'His Name(히즈 네임)']

,title,recommend,clicks,download,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,tag10,emotion
1357,His Name(히즈 네임),16,6359,3420,음악,배경음악,순수음악,WHITE,깔끔함,기타(ETC),어쿠스틱,드라마,DRAMA,느리게,우울


In [55]:
music = music.iloc[1:, :]

In [64]:
URL = 'https://gongu.copyright.or.kr/gongu/wrt/wrtCl/listWrtSound.do?menuNo=200020&pageIndex=1&searchWrd='
for mtitle, tag1, tag2, tag3 in tqdm(zip(list(music['title']), list(music['tag1']), list(music['tag2']), list(music['tag3']))):
    driver.get(URL + mtitle)
    time.sleep(2)
    titles = driver.find_elements_by_css_selector('ul > li > div:nth-child(1) > div.col-md-8 > span.tit > a')
    tags1 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(1)")
    tags2 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(2)")
    tags3 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(3)")
    pagein(mtitle, tag1, tag2, tag3, titles, tags1, tags2, tags3)
    time.sleep(1)
    try:
        driver.find_element_by_css_selector('#chkDownAt').click()
    except:
        print(mtitle)
        continue
    driver.find_element_by_css_selector('#downChkBtn').click()
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[-1])
    try:
        driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
    except:
        print(mtitle)
        driver.switch_to.window(driver.window_handles[0])
        continue        
    if driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span') == []:
        driver.switch_to.window(driver.window_handles[0])
        continue
    downloadmp3(driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span'),
                driver.find_elements_by_css_selector('#file_list > li > ul > li.input_chk'))
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(1)

16it [02:16,  8.83s/it]

소곰쟁이


17it [02:22,  7.98s/it]

쪼각빗


18it [02:25,  6.54s/it]

...

19it [02:31,  6.14s/it]

가을


20it [02:35,  5.52s/it]

꽃밭


21it [02:44,  7.81s/it]


In [32]:
URL = 'https://gongu.copyright.or.kr/gongu/wrt/wrtCl/listWrtSound.do?menuNo=200020&pageIndex=1&searchWrd='
music_titles = []
file_titles = []
for mtitle, tag1, tag2, tag3 in tqdm(zip(list(music['title']), list(music['tag1']), list(music['tag2']), list(music['tag3']))):
    driver.get(URL + mtitle)
    time.sleep(2)
    titles = driver.find_elements_by_css_selector('ul > li > div:nth-child(1) > div.col-md-8 > span.tit > a')
    tags1 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(1)")
    tags2 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(2)")
    tags3 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(3)")
    pagein(mtitle, tag1, tag2, tag3, titles, tags1, tags2, tags3)
    time.sleep(1)
    try:
        driver.find_element_by_css_selector('#chkDownAt').click()
    except:
        music_titles.append(mtitle)
        file_titles.append(np.NaN)
        continue
    driver.find_element_by_css_selector('#downChkBtn').click()
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[-1])
    try:
        driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
    except:
        music_titles.append(mtitle)
        file_titles.append(np.NaN)
        driver.switch_to.window(driver.window_handles[0])
        continue        
    if driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span') == []:
        music_titles.append(mtitle)
        file_titles.append(np.NaN)
        driver.switch_to.window(driver.window_handles[0])
        continue
    cnt = 0
    for name in driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span'):
        if name.text[-3:] in ['mp3', 'wav']:
            music_titles.append(mtitle)
            file_titles.append(name.text)
            break
        cnt += 1
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(1)

44it [16:12, 22.10s/it]


TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.870
  (Session info: chrome=91.0.4472.77)


In [33]:
music_titles

['4. Keyboard Concerto No. 5 in F Minor BWV 1056 - Largo_ 키보드 협주곡 제 5번 F단조 BWV 1056. 제2악장 라르고_ballad',
 "9. Prelude, op.28-15(Rain Drop)_ 쇼팽 : 전주곡 15번 내림 라장조 작품번호 28-15 '빗방울'_ballad",
 '2. Frolicher Landmann_ 행복한 농부_ragtime',
 '27. Kinderszenen Op.15 - VII. Traumerei_ 어린이의 정경 중 꿈 작품번호 15_ballad',
 '97. Symphony No. 5 in Cminor, Op.67_ 교향곡 제5번 다단조 [운명] OP.67_string',
 '11. The Swan(Le Cygne)_ 백조_fast swing',
 '94. Carmen Prelude_ 카르멘 서곡_string',
 '12. The Swan(Le Cygne)_ 백조_ballad',
 "101. Serenade No.13 in G Major K525 'Eine Kleins Nachtmusik'_ 세레나데 제 13번 사장조 '아이네 클라이네 나흐트 무지크'_string",
 "102. Symphony No. 8 in B minor. D759 'Unfinished'_ 교향곡 8번 나단조 D759 '미완성'_string",
 '23. Minuet In G Major BWV Anh. II 114_ 미뉴에트 사장조 BWV Anh. II 114_fast swing',
 '15. Sarabande_ 사라방드_easy listening',
 '17. Nocturne Op.9-2_ 녹턴 작품번호 9-2_funk',
 '34. Ave Maria_ 아베 마리아_ballad',
 '31. Songs my mother taught me op.55-4_ 어머니가 가르쳐 주신 노래 작품노래 55-4_ballad',
 "29. Salut D'amour In E Major Op.12_ 사랑의 인사 마장조 작품번호 

In [42]:
music = music.iloc[44:,:]

In [48]:
driver.switch_to.window(driver.window_handles[0])

In [45]:
URL = 'https://gongu.copyright.or.kr/gongu/wrt/wrtCl/listWrtSound.do?menuNo=200020&pageIndex=1&searchWrd='
for mtitle, tag1, tag2, tag3 in tqdm(zip(list(music['title']), list(music['tag1']), list(music['tag2']), list(music['tag3']))):
    driver.get(URL + mtitle)
    time.sleep(2)
    titles = driver.find_elements_by_css_selector('ul > li > div:nth-child(1) > div.col-md-8 > span.tit > a')
    tags1 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(1)")
    tags2 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(2)")
    tags3 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(3)")
    pagein(mtitle, tag1, tag2, tag3, titles, tags1, tags2, tags3)
    time.sleep(1)
    try:
        driver.find_element_by_css_selector('#chkDownAt').click()
    except:
        music_titles.append(mtitle)
        file_titles.append(np.NaN)
        continue
    driver.find_element_by_css_selector('#downChkBtn').click()
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[-1])
    try:
        driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
    except:
        music_titles.append(mtitle)
        file_titles.append(np.NaN)
        driver.switch_to.window(driver.window_handles[0])
        continue        
    if driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span') == []:
        music_titles.append(mtitle)
        file_titles.append(np.NaN)
        driver.switch_to.window(driver.window_handles[0])
        continue
    cnt = 0
    for name in driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span'):
        if name.text[-3:] in ['mp3', 'wav']:
            music_titles.append(mtitle)
            file_titles.append(name.text)
            break
        cnt += 1
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(1)

106it [52:08, 29.51s/it]


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=91.0.4472.77)


In [50]:
music_titles

['4. Keyboard Concerto No. 5 in F Minor BWV 1056 - Largo_ 키보드 협주곡 제 5번 F단조 BWV 1056. 제2악장 라르고_ballad',
 "9. Prelude, op.28-15(Rain Drop)_ 쇼팽 : 전주곡 15번 내림 라장조 작품번호 28-15 '빗방울'_ballad",
 '2. Frolicher Landmann_ 행복한 농부_ragtime',
 '27. Kinderszenen Op.15 - VII. Traumerei_ 어린이의 정경 중 꿈 작품번호 15_ballad',
 '97. Symphony No. 5 in Cminor, Op.67_ 교향곡 제5번 다단조 [운명] OP.67_string',
 '11. The Swan(Le Cygne)_ 백조_fast swing',
 '94. Carmen Prelude_ 카르멘 서곡_string',
 '12. The Swan(Le Cygne)_ 백조_ballad',
 "101. Serenade No.13 in G Major K525 'Eine Kleins Nachtmusik'_ 세레나데 제 13번 사장조 '아이네 클라이네 나흐트 무지크'_string",
 "102. Symphony No. 8 in B minor. D759 'Unfinished'_ 교향곡 8번 나단조 D759 '미완성'_string",
 '23. Minuet In G Major BWV Anh. II 114_ 미뉴에트 사장조 BWV Anh. II 114_fast swing',
 '15. Sarabande_ 사라방드_easy listening',
 '17. Nocturne Op.9-2_ 녹턴 작품번호 9-2_funk',
 '34. Ave Maria_ 아베 마리아_ballad',
 '31. Songs my mother taught me op.55-4_ 어머니가 가르쳐 주신 노래 작품노래 55-4_ballad',
 "29. Salut D'amour In E Major Op.12_ 사랑의 인사 마장조 작품번호 

In [54]:
music = music.iloc[106:,:]

In [55]:
URL = 'https://gongu.copyright.or.kr/gongu/wrt/wrtCl/listWrtSound.do?menuNo=200020&pageIndex=1&searchWrd='
for mtitle, tag1, tag2, tag3 in tqdm(zip(list(music['title']), list(music['tag1']), list(music['tag2']), list(music['tag3']))):
    driver.get(URL + mtitle)
    time.sleep(2)
    titles = driver.find_elements_by_css_selector('ul > li > div:nth-child(1) > div.col-md-8 > span.tit > a')
    tags1 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(1)")
    tags2 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(2)")
    tags3 = driver.find_elements_by_css_selector("#contents > div.bbsList.style2 > ul > li > div:nth-child(3) > div.col-md-9 > div > ul > li:nth-child(3)")
    pagein(mtitle, tag1, tag2, tag3, titles, tags1, tags2, tags3)
    time.sleep(1)
    try:
        driver.find_element_by_css_selector('#chkDownAt').click()
    except:
        music_titles.append(mtitle)
        file_titles.append(np.NaN)
        continue
    driver.find_element_by_css_selector('#downChkBtn').click()
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[-1])
    try:
        driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
    except:
        music_titles.append(mtitle)
        file_titles.append(np.NaN)
        driver.switch_to.window(driver.window_handles[0])
        continue        
    if driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span') == []:
        music_titles.append(mtitle)
        file_titles.append(np.NaN)
        driver.switch_to.window(driver.window_handles[0])
        continue
    cnt = 0
    for name in driver.find_elements_by_css_selector('#file_list > li > ul > li.fname > span'):
        if name.text[-3:] in ['mp3', 'wav']:
            music_titles.append(mtitle)
            file_titles.append(name.text)
            break
        cnt += 1
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(1)

142it [41:45, 17.64s/it]


In [58]:
pd.DataFrame({'music_titles' : music_titles, 'file_titles' : file_titles}).to_csv('title.csv')

In [ ]:
# 제목 목록
driver.find_elements_by_css_selector('ul > li > div:nth-child(1) > div.col-md-8 > span.tit > a')[0].text

In [ ]:
# 저작권 동의 클릭
driver.find_element_by_css_selector('#chkDownAt').click()

In [ ]:
# 다운로드
driver.find_element_by_css_selector('#downChkBtn').click()

In [ ]:
# 창 스위치
driver.switch_to.window(driver.window_handles[-1])

In [ ]:
#iframe으로 전환
driver.switch_to.frame('dext5uploader_frame_dext5upload')

In [ ]:
# 다운로드 파일 이름 목록 & 체크박스
driver.find_elements_by_css_selector('#file_list > li:nth-child(1) > ul > li.fname > span')
driver.find_elements_by_css_selector('#file_list > li > ul > li.input_chk')

In [ ]:
# 다운로드 클릭
driver.find_element_by_css_selector('#button_download').click()